# Assignment 2: Building a Simple Search Index

In this assignment, we will build a simple search index, which we will use later for Boolean retrieval. The assignment tasks are again at the bottom of this document.

## Loading the Data

In [1]:
Summaries_file = 'data/aspirin__Summaries.pkl.bz2'
Abstracts_file = 'data/aspirin__Abstracts.pkl.bz2'

In [2]:
import pickle, bz2
from collections import namedtuple

Summaries = pickle.load( bz2.BZ2File( Summaries_file, 'rb' ) )

paper = namedtuple( 'paper', ['title', 'authors', 'year', 'doi'] )

for (id, paper_info) in Summaries.items():
    Summaries[id] = paper( *paper_info )
    
Abstracts = pickle.load( bz2.BZ2File( Abstracts_file, 'rb' ) )

Let's have a look at what the data looks like for our example paper:

In [3]:
Summaries[28792522]

paper(title='Biochemical characterization and anti-inflammatory properties of an isothiocyanate-enriched moringa (Moringa oleifera) seed extract.', authors=['Jaja-Chimedza A', 'Graf BL', 'Simmler C', 'Kim Y', 'Kuhn P', 'Pauli GF', 'Raskin I'], year=2017, doi='10.1371/journal.pone.0182658')

In [4]:
Abstracts[28792522]

'Moringa oleifera Lam. is a tropical plant, used for centuries as food and traditional medicine. The aim of this study was to develop, validate and biochemically characterize an isothiocyanate-enriched moringa seed extract (MSE), and to compare the anti-inflammatory effects of MSE-containing moringa isothiocyanate-1 (MIC-1) with a curcuminoid-enriched turmeric extract (CTE), and a material further enriched in its primary phytochemical, curcumin (curcumin-enriched material; CEM). MSE was prepared by incubating ground moringa seeds with water to allow myrosinase-catalyzed enzymatic formation of bioactive MIC-1, the predominant isothiocyanate in moringa seeds. Optimization of the extraction process yielded an extract of 38.9% MIC-1. Phytochemical analysis of MSE revealed the presence of acetylated isothiocyanates, phenolic glycosides unique to moringa, flavonoids, fats and fatty acids, proteins and carbohydrates. MSE showed a reduction in the carrageenan-induced rat paw edema (33% at 500 

## Some Utility Functions

We'll define some utility functions that allow us to tokenize a string into terms, perform linguistic preprocessing on a list of terms, as well as a function to display information about a paper in a nice way. Note that these tokenization and preprocessing functions are rather naive - you may have to make them smarter in a later assignment.

In [5]:
def tokenize(text):
    """
    Function that tokenizes a string in a rather naive way. Can be extended later.
    """
    return text.split(' ')

def preprocess(tokens):
    """
    Perform linguistic preprocessing on a list of tokens. Can be extended later.
    """
    result = []
    for token in tokens:
        result.append(token.lower())
    return result

print(preprocess(tokenize("Lorem ipsum dolor sit AMET")))

['lorem', 'ipsum', 'dolor', 'sit', 'amet']


In [6]:
from IPython.display import display, HTML
import re

def display_summary( id, show_abstract=False, show_id=True, extra_text='' ):
    """
    Function for printing a paper's summary through IPython's Rich Display System.
    Trims long author lists, and adds a link to the paper's DOI (when available).
    """
    s = Summaries[id]
    lines = []
    title = s.title
    if s.doi != '':
        title = '<a href=http://dx.doi.org/%s>%s</a>' % (s.doi, title)
    title = '<strong>' + title + '</strong>'
    lines.append(title)
    authors = ', '.join( s.authors[:20] ) + ('' if len(s.authors) <= 20 else ', ...')
    lines.append(str(s.year) + '. ' + authors)
    if (show_abstract):
        lines.append('<small><strong>Abstract:</strong> <em>%s</em></small>' % Abstracts[id])
    if (show_id):
        lines.append('[ID: %d]' % id)
    if (extra_text != ''):
         lines.append(extra_text)
    display( HTML('<br>'.join(lines)) )

display_summary(28792522)
display_summary(28792522, show_abstract=True)

## Creating our first index

We will now create an _inverted index_ based on the words in the abstracts of the papers in our dataset.

We will implement our inverted index as a **Python dictionary with terms as keys and posting lists as values**. For the posting lists, instead of using Python lists and then implementing the different operations on them ourselves, we will use **Python sets** and use the predefined set operations to process these posting "lists". This will also ensure that each document is added at most once per term. The use of Python sets is not the most efficient solution but will work for our purposes. (As an optional additional exercise, you can try to implement the posting lists as Python lists for this and the following assignments.)

Not every paper in our dataset has an abstract; we will only index papers for which an abstract is available.

In [7]:
from collections import defaultdict

inverted_index = defaultdict(set)

# This may take a while:
for (id, abstract) in Abstracts.items():
    for term in preprocess(tokenize(abstract)):
        inverted_index[term].add(id)

Let's see what's in the index for the example term 'network':

In [8]:
print(inverted_index['network'])

{8573952, 22148098, 21847045, 17670662, 16996875, 26856461, 28409870, 17524242, 18359317, 11076124, 7558686, 27919915, 25130541, 27069488, 7375411, 10602550, 18393143, 10577464, 26342455, 12843581, 28401726, 2522689, 24008261, 2661447, 22946893, 6800979, 26274912, 26062437, 27149936, 23831164, 23083132, 28936833, 10952836, 22517902, 24473230, 26846361, 28204189, 24931488, 17940128, 26173603, 8430245, 12916918, 19681476, 26478300, 24614630, 17714922, 26851562, 26752241, 18078964, 19122933, 18465536, 28522244, 14677771, 22978320, 25632017, 26559249, 28215062, 17205530, 26883867, 28304154, 26237215, 2238968, 15745323, 28464428, 20494125, 17279276, 28933424, 26739516, 25373502, 25717062, 20514633, 11170635, 22089038, 28842832, 16941406, 7718753, 17912165, 26660710, 28336490, 25891179, 20061035, 28681581, 27957624, 12175738, 10608000, 19862935, 28385176, 21781406, 19966367, 10740135, 26337707, 25016235, 28267439, 9212859, 26122183, 19544525, 27487187, 20346324, 23463387, 18349026, 20967400,

We can now use this inverted index to answer simple one-word queries, for example to show all papers that contain the word 'netherlands':

In [9]:
query_word = 'netherlands'
for i in inverted_index[query_word]:
    display_summary(i)

----------

# Tasks

**Your name:** ...

### Task 1

Construct a function called `and_query` that takes as input a single string, consisting of one or more words, and returns a list of matching documents. `and_query`, as its name suggests, should require that all query terms are present in the documents of the result list. Demonstrate the working of your function with an example (choose one that leads to fewer than 100 hits to not overblow this notebook file).

(You can use the `tokenize` and `preprocess` functions we defined above to tokenize and preprocess your query. You can also exploit the fact that the posting lists are [sets](https://docs.python.org/3/library/stdtypes.html#set), which means you can easily perform set operations such as union, difference and intersect on them.)

In [ ]:
# Add your code here

### Task 2

Construct a second function called `or_query` that works in the same way as `and_query` you just implemented, but returns documents that contain _at least one_ of the words in the query. Demonstrate the working of this second function also with an example (again, choose one that leads to fewer than 100 hits).

In [ ]:
# Add your code here

### Task 3

Show how many hits the query "to be or not to be" returns for your two query functions (`and_query` and `or_query`).

In [ ]:
# Add your code here

### Task 4

Given the nature of our dataset, how many of the documents from task 3 do you think are actually about the [Shakespeare quote](https://en.wikipedia.org/wiki/To_be%2C_or_not_to_be)? What could you do to better handle such queries? (You don't have to implement this yet!)

**Answer:** [Write your answer text here]

### Task 5

Why does `and_query('moringa seed extracts')` not return our example paper 28792522, even though it mentions moringa seed extracts in the abstract? (You do not have to implement anything to fix this yet!)

**Answer:** [Write your answer text here]

## Submission

Submit the answers to the assignment as a modified version of this Notebook file (file with `.ipynb` extension) that includes your code and your answers via Canvas. Don't forget to add your name, and remember that the assignments have to be done individually and group submissions are **not allowed**.